### Objetivo del laboratorio
Aplicar técnicas de limpieza, transformación y manejo estructurado de datos utilizando Spark DataFrames en Databricks, sobre un caso realista del negocio de farmacias. Los alumnos aprenderán a:

- Identificar y depurar datos inválidos o incompletos.
- Crear columnas derivadas con funciones nativas de Spark.
- Realizar agregaciones y uniones eficientes entre múltiples fuentes.
- Eliminar duplicados para asegurar la integridad de la información.
- Guardar los resultados en formato Delta dentro de Unity Catalog siguiendo buenas prácticas de gobernanza y control de acceso.

### Procedimiento

1. Creamos catalogo y schema
2. Definimos rutas de origen
3. Leemos archivos de origen
4. Limpiamos a los diferentes datasets
5. Creamos columnas derivadas
6. Aplicamos joins para generar nuevos datasets
7. Aplicamos agregaciones para nuevos datasets
8. Escribimos en formato delta

In [0]:
from pyspark.sql.functions import lower, col, trim, when, coalesce, lit, round, to_date, year, month, sum, countDistinct, avg
from pyspark.sql.types import DoubleType

In [0]:
%sql
CREATE CATALOG gold_farmacias

In [0]:
catalog = "dmc"
schema_name = "gold_farmacias"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{schema_name}")

In [0]:
path_clientes = "/Volumes/dmc/gold_farmacias/source/Sesion_05/clientes.csv"
path_productos = "/Volumes/dmc/gold_farmacias/source/Sesion_05/productos.csv"
path_sucursales = "/Volumes/dmc/gold_farmacias/source/Sesion_05/sucursales.csv"
path_ventas = "/Volumes/dmc/gold_farmacias/source/Sesion_05/ventas.csv"

df_clientes = spark.read.option("header", True).option("inferSchema", True).csv(path_clientes)
df_productos = spark.read.option("header", True).option("inferSchema", True).csv(path_productos)
df_sucursales = spark.read.option("header", True).option("inferSchema", True).csv(path_sucursales)
df_ventas = spark.read.option("header", True).option("inferSchema", True).csv(path_ventas)

In [0]:
df_productos_clean = (
    df_productos
    .withColumn("nombre_producto", trim(lower(col("nombre_producto"))))
)

In [0]:
df_clientes_clean = (
    df_clientes
    .withColumn("email", when(col("email").contains("@"), col("email")))
    .withColumn("email", coalesce(col("email"), lit("-")))
)

In [0]:
df_ventas_clean = (
    df_ventas
    .dropna(subset=["cantidad"])
    .filter(col("precio_unitario") > 0)
)

![](/Volumes/dmc/gold_farmacias/source/Diagrama_AdventureWorksLT_01.png)

In [0]:
df_ventas_procesada = (
    df_ventas_clean
    .withColumn("monto_total", round(col("cantidad").cast(DoubleType()) * col("precio_unitario").cast("double"), 2))
    .withColumn("fecha_venta", to_date(col("fecha_venta")))
    .withColumn("anio_venta", year(col("fecha_venta")))
    .withColumn("mes_venta", month(col("fecha_venta")))
)

In [0]:
df_join = (
    df_ventas_procesada.alias("vp")
    .join(
        df_clientes_clean.alias("cc").select("id_cliente", "nombre", "apellido", "email"),
        col("vp.id_cliente") == col("cc.id_cliente"),
        "left"
    )
    .join(
        df_productos_clean.alias("pc").select("id_producto", "nombre_producto", "categoria", "marca", "costo_unitario"),
        col("vp.id_producto") == col("pc.id_producto"),
        "left"
    )
    .join(
        df_sucursales.alias("s").select("id_sucursal", "ciudad", "distrito", "nombre_sucursal"),
        col("vp.id_sucursal") == col("s.id_sucursal"),
        "left"
    )
    .select(
        col("vp.id_venta").alias("id_venta"),
        col("vp.fecha_venta").alias("fecha_venta"),
        col("vp.id_cliente").alias("id_cliente"),
        col("cc.nombre").alias("nombre"),
        col("cc.apellido").alias("apellido"),
        col("cc.email").alias("email"),
        col("vp.id_producto").alias("id_producto"),
        col("pc.nombre_producto").alias("nombre_producto"),
        col("pc.categoria").alias("categoria"),
        col("pc.marca").alias("marca"),
        col("pc.costo_unitario").alias("costo_unitario"),
        col("vp.id_sucursal").alias("id_sucursal"),
        col("s.ciudad").alias("ciudad"),
        col("s.distrito").alias("distrito"),
        col("s.nombre_sucursal").alias("nombre_sucursal"),
        col("vp.cantidad").alias("cantidad"),
        col("vp.precio_unitario").alias("precio_unitario"),
        col("vp.monto_total").alias("monto_total"),
        # (col("vp.cantidad") * col("vp.precio_unitario")).alias("monto_total_2"),
        col("vp.anio_venta").alias("anio_venta"),
        col("vp.mes_venta").alias("mes_venta")        
    )
).dropDuplicates(["id_venta"])

In [0]:
df_categoria_ticket = (
    df_join
    .withColumn("categoria_ticket", 
                when(col("monto_total") >= 200, lit("Alto"))
                .when(col("monto_total") >= 80, lit("Medio"))
                .otherwise(lit("Bajo"))
    )
)

In [0]:
df_ventas_ciudad_anio = (
    df_join
    .groupBy("ciudad", "anio_venta")
    .agg(
        sum("monto_total").alias("venta_total"),
        countDistinct("id_venta").alias("num_ventas")
    )
)

In [0]:
df_ventas_cliente = (
    df_join
    .groupBy("id_cliente")
    .agg(
        avg("monto_total").alias("venta_promedio"),
        countDistinct("id_venta").alias("num_ventas")
    )
)

In [0]:
df_categoria_ventas = (
    df_categoria_ticket
    .groupBy("categoria_ticket")
    .agg(
        sum("monto_total").alias("ventas"),
        countDistinct("id_venta").alias("num_ventas")
    )
)

In [0]:
tbl_detalle = f"{catalog}.{schema_name}.ventas_detalle"
tbl_ciudad = f"{catalog}.{schema_name}.ventas_ciudad_anio"
tbl_promedio = f"{catalog}.{schema_name}.ventas_promedio"
tbl_categoria = f"{catalog}.{schema_name}.ventas_categoria"

In [0]:
df_join.write.format("delta").mode("overwrite").saveAsTable(tbl_detalle)

In [0]:
df_join.write.format("delta").mode("overwrite").partitionBy("anio_venta", "mes_venta").save("/Volumes/dmc/gold_farmacias/source/ventas_detalle/")

In [0]:
df_ventas_ciudad_anio.write.format("delta").mode("overwrite").saveAsTable(tbl_ciudad)

In [0]:
df_ventas_cliente.write.format("delta").mode("overwrite").saveAsTable(tbl_promedio)

In [0]:
df_categoria_ventas.write.format("delta").mode("overwrite").saveAsTable(tbl_categoria)

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_detalle

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_ciudad_anio

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_promedio

In [0]:
%sql
select * from dmc.gold_farmacias.ventas_categoria